In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [3]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


[Info](https://whimsical.com/simple-local-rag-workflow-39kToR3yNf7E8kY4sS2tjV)

In [4]:
import os
import requests

dataDir = "./data/human-nutrition-text.pdf"

if not os.path.exists(dataDir):
    print(f"Doesn't exist {dataDir}")
else:
    print(f"File exists {dataDir}")


File exists ./data/human-nutrition-text.pdf


In [19]:
import fitz 
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    return text.replace('\n', ' ').strip()


def read_pdf(path: str) -> list[dict]:
    doc = fitz.open(path)
    #print(f"Number of pages: {len(doc)}")
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_text.append({"page_number":page_number,
                               "page_char_count":len(text),
                               "page_word_count":len(text.split(" ")),
                               "page_sentence_count":len(text.split(". ")),
                               "page_token_count":len(text) / 4,
                               "page_text":text})
    return pages_and_text

In [ ]:
pages_and_text = read_pdf(path = dataDir)


0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 1,
  'page_token_count': 7.25,
  'page_text': 'Human Nutrition: 2020 Edition'},
 {'page_number': 1,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'page_token_count': 0.0,
  'page_text': ''},
 {'page_number': 2,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count': 1,
  'page_token_count': 80.0,
  'page_text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'},
 {'page_number': 3,
  'page_char_count': 212,
  'page_word_count': 32,
  'page_sentence_count': 1,
  'page_token_count': 53.0,
  'page_text': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Scie

In [22]:

pages_and_text[:2]

[{'page_number': 0,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 1,
  'page_token_count': 7.25,
  'page_text': 'Human Nutrition: 2020 Edition'},
 {'page_number': 1,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'page_token_count': 0.0,
  'page_text': ''}]

In [23]:
import random

random.sample(pages_and_text, 2)

[{'page_number': 808,
  'page_char_count': 213,
  'page_word_count': 36,
  'page_sentence_count': 2,
  'page_token_count': 53.25,
  'page_text': 'An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=432    Pacific Based Dietary Guidelines  |  767'},
 {'page_number': 644,
  'page_char_count': 1589,
  'page_word_count': 234,
  'page_sentence_count': 28,
  'page_token_count': 397.25,
  'page_text': 'Shi, J., Arunasalam, K., Yeung, D., Kakuda, Y., Mittal, G., & Jiang,  Y. (2004). Saponins from Edible Legumes: Chemistry, Processing,  and Health Benefits. Journal of Medicinal Food, 7(1), 67–78.  https://doi.org/10.1089/109662004322984734  Thompson, G. R., & Grundy, S. M. (2005). History and Development  of Plant Sterol and Stanol Esters for Cholesterol-Lowering  Purposes. The American Journal of Cardiology, 96(1, Supplement),  3–9. https://doi.org/10.1016/j.amjcard.2005.03.013  Xu

In [24]:
import pandas as pd 

df = pd.DataFrame(pages_and_text)

df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_text
0,0,29,4,1,7.25,Human Nutrition: 2020 Edition
1,1,0,1,1,0.00,
2,2,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,3,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,4,797,147,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [25]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,603.50,1148.00,199.50,10.52,287.00
std,348.86,560.38,95.83,6.55,140.10
min,0.00,0.00,1.00,1.00,0.00
25%,301.75,762.00,134.00,5.00,190.50
50%,603.50,1231.50,216.00,10.00,307.88
75%,905.25,1603.50,272.00,15.00,400.88
max,1207.00,2308.00,430.00,39.00,577.00


In [ ]:
# from spacy.lang.uk import Ukrainian
from spacy.lang.en import English

